In [1]:
import numpy as np 
import pandas as pd 
import re
import fasttext
import pickle

OUT_PATH = './out/15_02_22_CRH_VAL_FastText.pkl'

# Export mutlilabel classification prediction 

Takes a pandas dataframe with sigmoid values and export as a pd dataframe with all labels for all terms in the files, and get the genders

In a second step, get all the couples term/label per file in a pandas Dataframe, keeping track of the proportion 

In a third step : get FasText embeddings for all terms keep them in a pandas Dataframe. 

In [2]:
# Get the results of the classification step: 

model_name = '20220209T092729_expe_pheno_classifier_camembert_l_umls_fr_local'

result_path = f'./out/final_CRH_VAL_PHENO_pred_{model_name}.csv'
#result_path = 'final_test_SUB_EXAM_20210701T120111_classifier_camembert_umls_fr_mesh_mdr_val_0.1_deft.csv'
thresholds_path = f'./out/thresholds_{model_name}.csv'
labels_path = f'./out/labels_{model_name}.csv'

thresholds = np.loadtxt(thresholds_path, delimiter=",")
print(f'Thresholds loaded from {thresholds_path}')
with open(labels_path, 'r') as f_out:
    label_names = f_out.readline().strip().split(',')
print(f'Label names loaded from {labels_path}')
    
result_df = pd.read_csv(result_path)
if "Unnamed: 0" in result_df.columns:
    del(result_df["Unnamed: 0"])

print(f'Results loaded from {result_path}')

labels_from_results = [l for l in result_df.columns if l.startswith('label_')]
assert labels_from_results == label_names

result_data = result_df[label_names].to_numpy()
assert len(result_data[0]) == len(label_names) == len(thresholds)
# if all thresholds are the same, keep only one value
if len(set(thresholds)) == 1:
    thresholds = thresholds[0]



Thresholds loaded from ./out/thresholds_20220209T092729_expe_pheno_classifier_camembert_l_umls_fr_local.csv
Label names loaded from ./out/labels_20220209T092729_expe_pheno_classifier_camembert_l_umls_fr_local.csv
Results loaded from ./out/final_CRH_VAL_PHENO_pred_20220209T092729_expe_pheno_classifier_camembert_l_umls_fr_local.csv


In [3]:
result_df = pd.read_csv(result_path)

In [4]:
# Convert them in a dataframe

seuils = [0.75]
#seuils = [0.90]
for seuil in seuils : 
    print("\n seuil =", seuil)
    # Apply thresholds on data
    result_data_th = result_data > seuil
    result_df[label_names] = result_data_th    
    df2 = result_df.copy()

df2.sample(5)


 seuil = 0.75


,Unnamed: 0,term,source,label_ORL,label_blessures,label_cardiovasculaires,label_chimiques,label_digestif,label_endocriniennes,label_etatsosy,...,label_nutritionnelles,label_oeil,label_osteomusculaires,label_parasitaires,label_peau,label_respiratoire,label_stomatognathique,label_tumeur,label_urogen,label_virales
8608,8608,vih,CRH_val_sample_24.ann,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
53,53,syndrome inflammatoire,NEW_CRH_val_sample_62.ann,False,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False
9212,9212,aggravation pulmonaire,CRH_val_sample_182.ann,False,False,False,False,False,False,True,...,False,False,False,False,False,True,False,False,False,False
5584,5584,sensation de constriction thoracique,CRH_val_sample_98.ann,False,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False
590,590,fausse couche spontanée précoce,CRH_val_sample_373.ann,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False


In [5]:
new_df = df2[df2==1].stack().reset_index().drop(0,1)
new_df

new_df ['term'] = range(len(new_df))
new_df ['source'] = range(len(new_df))

for i in range(len(new_df)):
    idx_init = new_df['level_0'][i]
    new_df['term'][i] = df2['term'][idx_init]
    new_df['source'][i] = df2['source'][idx_init]

new_df.sample(5)

/export/home/cse200093/.user_conda/miniconda/envs/Expe_Pheno/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.
/export/home/cse200093/.user_conda/miniconda/envs/Expe_Pheno/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/export/home/cse200093/.user_conda/miniconda/envs/Expe_Pheno/lib/python3.7/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-

,level_0,level_1,term,source
24666,17919,label_respiratoire,tvo sévère,CRH_val_sample_330.ann
9896,7186,label_oeil,nystagmus horizontal gauche persistant,CRH_val_sample_318.ann
18597,13448,label_etatsosy,f,CRH_val_sample_193.ann
971,691,label_peau,syndrome sec buccal,CRH_val_sample_373.ann
23665,17194,label_genetique,doublement cérébrolésé,CRH_val_sample_7.ann


In [6]:
new_df_corr = new_df.replace('label_', '', regex = True)
new_df_corr

,level_0,level_1,term,source
0,0,etatsosy,f,NEW_CRH_val_sample_62.ann
1,1,nerveux,tétraparésie,NEW_CRH_val_sample_62.ann
2,2,immunitaire,myosite nécrosante,NEW_CRH_val_sample_62.ann
3,2,nerveux,myosite nécrosante,NEW_CRH_val_sample_62.ann
4,2,osteomusculaires,myosite nécrosante,NEW_CRH_val_sample_62.ann
...,...,...,...,...
25815,18813,digestif,colite ischémique,CRH_val_sample_345.ann
25816,18814,nutritionnelles,carence en vitamine d,CRH_val_sample_345.ann
25817,18815,etatsosy,carence en vitamine d supplémentée,CRH_val_sample_345.ann
25818,18816,endocriniennes,hypothyroïdie fruste,CRH_val_sample_345.ann


In [8]:
# Get f and m terms to get the gender info 
df_gender = new_df_corr[(new_df_corr['term'] == 'f') | (new_df_corr['term'] == 'm')]
df_gender.sample(5)

,level_0,level_1,term,source
14113,10287,etatsosy,f,CRH_val_sample_84.ann
6232,4507,etatsosy,f,NEW_CRH_val_sample_74.ann
2023,1479,etatsosy,f,CRH_val_sample_222.ann
12889,9411,hemopathies,m,NEW_CRH_val_sample_76.ann
3863,2796,etatsosy,f,NEW_CRH_val_sample_86.ann


In [9]:
del(df_gender['level_0'])
del(df_gender['level_1'])
df_gender.columns = ['gender', 'source']
df_gender.sample(5)

,gender,source
8617,m,CRH_val_sample_332.ann
10598,f,CRH_val_sample_64.ann
24189,m,CRH_val_sample_103.ann
17927,m,CRH_val_sample_304.ann
10294,f,NEW_CRH_val_sample_75.ann


In [10]:
len(df_gender), len(set(df_gender.source.tolist()))

(944, 242)

In [11]:
# get files without gender detection :
no_g = set(new_df_corr.source.tolist()) - set(df_gender.source.tolist())

In [12]:
# some patients have more than one gender detected : choose the gender that appears more times 
import collections

df_f = df_gender[df_gender['gender'] =='f']
df_m = df_gender[df_gender['gender'] =='m']
# get set of file names with two genders detected :
both_g = set(df_f.source.tolist()) & set(df_m.source.tolist())
counter_g = dict()
true_g = dict()
for file in both_g:
    # get the list of genders associated to this file 
    genders = df_gender[df_gender['source']== file].gender.tolist()
    counter_g[file] = collections.Counter(genders)
    if counter_g[file]['m'] >= counter_g[file]['f']:
        true_g[file] = 'm'
    else: 
        true_g[file] = 'f'
    # drop this from df_gender 
    df_gender = df_gender.drop(df_gender[df_gender['source'] == file].index)

# counter_g, true_g

In [13]:
# build the final gender dataset to be merged later on:
data_g = df_gender.values.tolist()
for file in both_g:
    data_g.append([true_g[file], file])
    
# given our population, by default, patient with no gender detected will be assigned to the female gender: 
for file_b in no_g:
    data_g.append(['f', file_b])

# convert the data into a dataframe 
columns = ['gender', 'source']
df_gender_corr = pd.DataFrame(data=list(data_g), columns=columns)
df_gender_corr = df_gender_corr.drop_duplicates().reset_index(drop=True)
len(df_gender_corr)

256

In [14]:
new_df_corr.drop(new_df_corr[(new_df_corr.term == 'f') | (new_df_corr.term == 'm')].index, inplace = True)

In [15]:
df_final = pd.merge(df_gender_corr, new_df_corr, on='source')
df_check = df_final.groupby('source')['level_1'].nunique()
len(df_check)

256

In [16]:
df_final
del(df_final['level_0'])

In [17]:
df_final.columns = ['gender', 'source','label', 'term']
df_final = df_final.replace('NEW', 'n', regex = True)
df_final.sample(5)

,gender,source,label,term
17763,f,n_CRH_val_sample_37.ann,etatsosy,douleur enthésopathique à la palpation
12906,f,CRH_val_sample_84.ann,hemopathies,thrombopénie
326,f,CRH_val_sample_191.ann,immunitaire,sclérodermie systémique
20691,m,CRH_val_sample_253.ann,etatsosy,épaississement pariétal de l'artère carotide c...
13756,m,CRH_val_sample_36.ann,peau,lupus


# Last step : Get the FastText embeddings of extracted and classified terms

In [18]:
model = fasttext.load_model("/export/home/cse200093/output/output.bin")

In [19]:
df_final.values[0,3]

'tétraparésie'

In [20]:
data = []
for i in range(len(df_final.values)):
    data.append([df_final.values[i,1],df_final.values[i,0], df_final.values[i,3],
                 model.get_word_vector(df_final.values[i,3]), 
                df_final.values[i,2]])

In [21]:
# save this dataset in a pd dataFrame 
df = pd.DataFrame(data)

In [22]:
df.columns = ['source', 'gender','term', 'fastext_embeddings', 'label']
df.sample(5)

,source,gender,term,fastext_embeddings,label
24063,n_CRH_val_sample_40.ann,f,gonflement le matin de résolution spontanée,"[-0.95660895, -1.410214, -5.0094676, -11.56897...",etatsosy
23083,CRH_val_sample_345.ann,f,cryoglobulinémie positive à taux faible (36.5 ...,"[-15.836896, 1.6881723, -5.4751887, -22.773846...",hemopathies
19317,n_CRH_val_sample_2.ann,m,onychomycose,"[80.69458, 115.68712, -84.38579, -93.31562, 14...",infections
1715,CRH_val_sample_225.ann,m,vascularite des gros vaisseaux type takayasu,"[-17.924557, 6.3821354, -44.4702, 7.9283524, -...",cardiovasculaires
5826,CRH_val_sample_319.ann,m,vascularite nécrosante,"[-2.6332772, 25.422905, -12.015024, -2.1633248...",cardiovasculaires


In [23]:
# save 
df.to_pickle(OUT_PATH)

In [21]:
# Finaly, in order to keep track of the importance ("weight") of a label at the document scale
# get the ratio : #term_for_a_given_label/#total_term

# empty list of list to "append" each values
data_C = []
# get the set of all documents source
enum = set(my_df.source)
for i in enum:
    # get the index of all terms for one  given document: 
    idx_i = my_df[my_df['source'] == i].index.tolist()
    # get the list of corresponding labels
    labels = my_df['label'][idx_i].tolist()
    # and terms
    term = my_df['term'][idx_i].tolist()
    # and embeddings
    fastext_embeddings = my_df['fastext_embeddings'][idx_i].tolist()
    
    # compute the weight for each label between 0 and 1
    for j in range(len(idx_i)) : 
        l = labels[j]
        count_l = labels.count(l)
        #
        weight_l = count_l/len(idx_i)
        # print(i,term[j],"\n", l, weight_l)
        data_C.append([i, term[j],fastext_embeddings[j],l, weight_l])
        

my_df_C = pd.DataFrame(data_C)


In [22]:
my_df_C.columns = ['source', 'term', 'fastext_embeddings', 'label', 'label_weight']
my_df_C

,source,term,fastext_embeddings,label,label_weight
0,CRH_scl_sample_50.ann,sclérodermie systémique,"[1.3671794, 4.706537, -21.904682, -30.129189, ...",peau,0.111111
1,CRH_scl_sample_50.ann,atteinte digestive,"[7.0249844, -16.497305, -9.14085, -8.498046, 5...",digestif,0.129630
2,CRH_scl_sample_50.ann,atteinte digestive,"[7.0249844, -16.497305, -9.14085, -8.498046, 5...",etatsosy,0.490741
3,CRH_scl_sample_50.ann,tabagisme actif,"[16.970762, 18.212788, -26.59381, -25.507788, ...",chimiques,0.018519
4,CRH_scl_sample_50.ann,ulcère gastrique,"[8.748739, -6.0027266, -25.528395, -21.069235,...",digestif,0.129630
...,...,...,...,...,...
2538,CRH_sapl_sample_23.ann,lupus systémique à déterminisme cutané,"[-11.12191, -1.757302, -21.213293, -0.06043457...",immunitaire,0.250000
2539,CRH_sapl_sample_23.ann,lupus systémique à déterminisme cutané,"[-11.12191, -1.757302, -21.213293, -0.06043457...",peau,0.151786
2540,CRH_sapl_sample_23.ann,sapl,"[-20.912071, -10.4872265, -43.142654, -135.454...",immunitaire,0.250000
2541,CRH_sapl_sample_23.ann,lupus systémique à déterminisme cutané et arti...,"[-10.710101, 2.856835, -13.931923, -2.393613, ...",immunitaire,0.250000


In [23]:
import pickle
# save
my_df_C.to_pickle('./out/20220210_local_annot_accent_classif_FastTextEmb.pkl')

In [24]:
df_check = my_df_C.groupby('source')['term'].nunique()

In [26]:
len(df_check)

28